# Лекция №6

В данном уроке мы познакомимся с тем, как использваоть предопученные нейронные сети с помощью OpenCV

Видите ли, чтобы получить (правильные) прогнозы из глубоких нейронных сетей, вам сначала нужно предварительно обработать ваши данные.

В контексте глубокого обучения и классификации изображений эти задачи предварительной обработки обычно включают в себя:
1. Среднее вычитание
2. Масштабирование по некоторому фактору

Новый модуль OpenCV для глубокой нейронной сети (**dnn**) содержит две функции, которые можно использовать для предварительной обработки изображений и подготовки их к классификации с помощью предварительно обученных моделей глубокого обучения.

В сегодняшнем посте мы разберем функции предварительной обработки OpenCV **cv2.dnn.blobFromImage** и **cv2.dnn.blobFromImages** и поймем, как они работают.

## Глубокое изучение: как работает blobFromImage в OpenCV
OpenCV предоставляет две функции, облегчающие предварительную обработку изображений для глубокой классификации обучения:
* cv2.dnn.blobFromImage
* cv2.dnn.blobFromImages

Эти две функции выполняют:
1. среднее вычитание
2. пересчет
3. при желании обмен каналов


Дальше мы рассмотрим:
1. среднее вычитание и масштабирование
2. сигнатуру функции каждой функции предварительной обработки глубокого обучения
3. изучим эти методы подробно
4. применим функции глубокого обучения OpenCV к набору входных изображений.

## Глубокое обучение и среднее вычитание

<img src="img/blob_from_images_mean_subtraction.jpg" alt="Drawing" style="width: 600px;"/>
На этом рисунке представлено среднее вычитание, где среднее значение RGB (в центре) было вычислено из набора данных изображений и вычтено из исходного изображения (слева), что привело к выходному изображению (справа).

Прежде чем мы углубимся в объяснение функций предварительной обработки глубокого обучения OpenCV, нам сначала необходимо понять среднее вычитание. Среднее вычитание используется для борьбы с изменениями освещенности входных изображений в нашем наборе данных. Поэтому мы можем рассматривать среднее вычитание как метод, используемый для помощи нашим сверточным нейронным сетям.

Прежде чем мы начнем тренировать нашу глубокую нейронную сеть, мы сначала вычисляем среднюю интенсивность пикселей по всем изображениям в обучающем наборе для каждого из красного, зеленого и синего каналов.

Это означает, что в итоге мы получим три переменные:$$\mu_R,\ \mu_G , \ \mu_B.$$


Обычно результирующие значения представляют собой 3-х кортеж, состоящий из среднего значения красного, зеленого и синего каналов соответственно.

Например, средними значениями для обучающего набора ImageNet являются $R = 103.93$, $G = 116.77$ и $B = 123.68$.

Когда мы готовы передать изображение через нашу сеть (для обучения или тестирования), мы вычитаем среднее значение $\mu$ из каждого входного канала входного изображения:
$$
R = R - \mu_R\\
G = G - \mu_G\\
B = B - \mu_B\\
$$
У нас также может быть коэффициент масштабирования $\sigma$, который добавляет нормализацию:
$$
R = (R - \mu_R) / \sigma\\
G = (G - \mu_G) / \sigma\\
B = (B - \mu_B) / \sigma\\
$$
Значение $\sigma$ может быть стандартным отклонением по обучающему набору. Тем не менее, $\sigma$ также может быть установлен вручную (по сравнению с вычисленным), чтобы масштабировать пространство входного изображения в конкретный диапазон - это действительно зависит от архитектуры, от того, как была обучена сеть.

Важно отметить, что не все архитектуры с глубоким обучением выполняют среднее вычитание и масштабирование! Перед предварительной обработкой изображений обязательно прочитайте соответствующую публикацию/документацию по используемой вами глубокой нейронной сети.

Как вы обнаружите в своем пути глубокого обучения, некоторые архитектуры выполняют только среднее вычитание (тем самым устанавливая $\sigma = 1$). Другие архитектуры выполняют как среднее вычитание, так и масштабирование. Всегда проверяйте соответствующую публикацию, которую вы используете.

## Функции OpenCV blobFromImage и blobFromImages
Начнем с обращения к [официальной документации OpenCV](https://docs.opencv.org/trunk/d6/d0f/group__dnn.html#ga33d1b39b53a891e98a654fdeabba22eb) для **cv2.dnn.blobFromImage**:

* *[blobFromImage] создает 4-мерное пятно из изображения. Опционально изменяет размеры и обрезает изображение по центру, вычитает средние значения, масштабирует значения по коэффициенту масштабирования, меняет синий и красный каналы.

Неформально, BLOB-объект - это просто (потенциально коллекция) изображений с одинаковыми пространственными размерами (то есть, шириной и высотой), одинаковой глубиной (числом каналов), которые должны быть предварительно обработаны одинаковым образом.

Функции **cv2.dnn.blobFromImage** и **cv2.dnn.blobFromImages** практически идентичны.

Давайте начнем с проверки подписи функции **cv2.dnn.blobFromImage** ниже:

**blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)**

1. **image** : это входное изображение, которое мы хотим предварительно обработать, прежде чем передать его через нашу глубокую нейронную сеть для классификации.

2. **scalefactor** : после выполнения среднего вычитания мы можем по желанию масштабировать изображения. По умолчанию это значение равно 1,0 (то есть без масштабирования), но мы также можем указать другое значение. Также важно отметить, что **scalefactor** должен быть равен $1 / \sigma$, поскольку мы фактически умножаем входные каналы (после среднего вычитания) на **scalefactor**.

3. **size** : здесь мы предоставляем пространственный размер, который ожидает сверточная нейронная сеть. Для большинства современных нейронных сетей это $224\times224$, $227\times227$ или $299\times299$.

4. **mean** : это наши средние значения вычитания. Они могут быть 3-мя кортежами RGB-средств или могут быть одним значением, и в этом случае предоставленное значение вычитается из каждого канала изображения. Если вы выполняете среднее вычитание, убедитесь, что вы поставили 3-кортеж в порядке **(R, G, B)**, особенно при использовании поведения **swapRB = True** по умолчанию.

5. **swapRB** : OpenCV предполагает, что изображения имеют порядок каналов **BGR**; однако среднее значение предполагает, что мы используем порядок RGB. Чтобы устранить это расхождение, мы можем поменять местами каналы **R** и **B** на изображении, установив для этого значения значение **True**. По умолчанию OpenCV выполняет этот обмен каналов для нас.
Функция **cv2.dnn.blobFromImage** возвращает BLOB-объект, который является нашим входным изображением после среднего вычитания, нормализации и переключения каналов.

Функция **cv2.dnn.blobFromImages** точно такая же:

**blob = cv2.dnn.blobFromImages(images, scalefactor=1.0, size, mean, swapRB=True)**

Единственное исключение состоит в том, что мы можем передавать несколько изображений, что позволяет нам пакетно обрабатывать набор изображений.

Если вы обрабатываете несколько изображений, обязательно используйте функцию **cv2.dnn.blobFromImages**, так как при этом меньше затрат на вызов функции и вы сможете быстрее обрабатывать изображения.

## Глубокое обучение с функцией OpenCV blobFromImage
Теперь, когда мы изучили функции **blobFromImage** и **blobFromImages**, давайте применим их к нескольким образцам изображений, а затем передадим их через сверточную нейронную сеть для классификации.

В качестве предварительного условия вам необходима версия OpenCV 3.3.0 как минимум. NumPy - это зависимость от привязок Python в OpenCV, а imutils $-$ пакет удобных функций, доступных на GitHub и в индексе пакетов Python.

Пакет **imutils** можно установить через pip:

In [2]:
!pip install imutils

  Running setup.py bdist_wheel for imutils: started
  Running setup.py bdist_wheel for imutils: finished with status 'done'
  Stored in directory: C:\Users\user\AppData\Local\pip\Cache\wheels\b2\40\59\139d450e68847ef2f27d876d527b13389dac23df0f66526b5d
Successfully built imutils


mkl-random 1.0.1 requires cython, which is not installed.
fastai 1.0.18 requires jupyter, which is not installed.
fastai 1.0.18 requires torchvision-nightly, which is not installed.
tensorflow 1.10.0 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.4 which is incompatible.
tensorflow 1.10.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.4.3 which is incompatible.
spacy 2.0.16 has requirement regex==2018.01.10, but you'll have regex 2018.8.29 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
# import the necessary packages
from imutils import paths
import numpy as np
import cv2

In [4]:
# load the class labels from disk
PATH_TXT = '..\\src\\blob-from-images\\synset_words.txt'
rows = open(PATH_TXT).read().strip().split("\n")
classes = [r[r.find(" ") + 1:].split(",")[0] for r in rows]
 
# load our serialized model from disk
PATH_PROTOTXT = '..\\src\\blob-from-images\\bvlc_googlenet.prototxt'
PATH_MODEL = '..\\src\\blob-from-images\\bvlc_googlenet.caffemodel'
net = cv2.dnn.readNetFromCaffe(PATH_PROTOTXT, PATH_MODEL)
 
# grab the paths to the input images
imagePaths = sorted(list(paths.list_images('..\\src\\blob-from-images\\images')))

In [5]:
imagePaths

['..\\src\\blob-from-images\\images\\beer.png',
 '..\\src\\blob-from-images\\images\\brown_bear.png',
 '..\\src\\blob-from-images\\images\\keyboard.png',
 '..\\src\\blob-from-images\\images\\monitor.png',
 '..\\src\\blob-from-images\\images\\space_shuttle.png']

Сначала мы импортируем imutils, numpy и cv2 **(строки 2-4)**.

Сначала мы читаем synset_words.txt (метки классов ImageNet) и извлекаем классы, наши метки классов.

Чтобы загрузить модель нашей модели с диска, мы используем функцию DNN **cv2.dnn.readNetFromCaffe** и указываем bvlc_googlenet.prototxt в качестве параметра имени файла, а bvlc_googlenet.caffemodel в качестве фактического файла модели.

*__Примечание.__ Ресурсы для этой части кода вы можете найти в папке src на [гитхабе курса](https://github.com/Keleas/CV_WOS).*

Наконец, мы берем пути к входным изображениям. 

Далее мы загрузим образы с диска и предварительно обработаем их с помощью **blobFromImage**:

In [3]:
# (1) load the first image from disk, (2) pre-process it by resizing
# it to 224x224 pixels, and (3) construct a blob that can be passed
# through the pre-trained network
image = cv2.imread(imagePaths[0])
resized = cv2.resize(image, (224, 224))
blob = cv2.dnn.blobFromImage(resized, 1, (224, 224), (104, 117, 123))
print("First Blob: {}".format(blob.shape))

First Blob: (1, 3, 224, 224)


В этом блоке мы сначала загружаем image, а затем изменяем его размер до $224\times224$, необходимых размеров входного изображения для **GoogLeNet**.

Мы вызываем **cv2.dnn.blobFromImage**, который, как указано в предыдущем разделе, создаст 4-мерный blob -объект для использования в нашей нейронной сети.

Давайте напечатаем форму нашего blob-объекта, чтобы позже мы могли проанализировать его.

Далее, мы будем кормить blob через GoogLeNet:

In [1]:
# set the input to the pre-trained deep learning network and obtain
# the output predicted probabilities for each of the 1,000 ImageNet
# classes
net.setInput(blob)
preds = net.forward()

preds

NameError: name 'net' is not defined

In [ ]:
# sort the probabilities (in descending) order, grab the index of the
# top predicted label, and draw it on the input image
idx = np.argsort(preds[0])[::-1][0]
text = "Label: {}, {:.2f}%".format(classes[idx], preds[0][idx] * 100)
cv2.putText(image, text, (5, 25),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

In [ ]:
# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Мы пропускаем BLOB через сеть и получаем прогнозы, preds.

Затем мы сортируем preds с наиболее достоверными прогнозами в начале списка и генерируем текст метки для отображения на изображении. Текст метки состоит из метки класса и процентного значения прогноза для верхнего прогноза.

Оттуда мы пишем text метки в верхней части image, а затем отображаем image на экране и ждем нажатия клавиши, прежде чем двигаться дальше.

_Теперь пришло время использовать множественную форму функции **blobFromImage**._

Здесь мы сделаем (почти) то же самое, за исключением того, что вместо этого создадим и заполните список images, а затем передадим список в качестве параметра **blobFromImages**:

In [5]:
# initialize the list of images we'll be passing through the network
images = []
 
# loop over the input images (excluding the first one since we
# already classified it), pre-process each image, and update the
# `images` list
for p in imagePaths[1:]:
    image = cv2.imread(p)
    image = cv2.resize(image, (224, 224))
    images.append(image)
 
# convert the images list into an OpenCV-compatible blob
blob = cv2.dnn.blobFromImages(images, 1, (224, 224), (104, 117, 123))
print("Second Blob: {}".format(blob.shape))

Second Blob: (4, 3, 224, 224)


Сначала мы инициализируем наш список images, а затем, используя imagePaths, читаем, изменяем размер и добавляем image в список.

Используя нарезку списка, мы пропустили первое изображение из imagePaths.

Оттуда мы передаем images в **cv2.dnn.blobFromImages** в качестве первого параметра. Все остальные параметры для **cv2.dnn.blobFromImages** идентичны **cv2.dnn.blobFromImage** выше.

Для последующего анализа мы печатаем **blob.shape**.

Затем мы пропустим blob через GoogLeNet и напишем метку класса и прогноз в верхней части каждого изображения:

In [6]:
# set the input to our pre-trained network and obtain the output
# class label predictions
net.setInput(blob)
preds = net.forward()
 
# loop over the input images
for (i, p) in enumerate(imagePaths[1:]):
    # load the image from disk
    image = cv2.imread(p)

    # find the top class label from the `preds` list and draw it on
    # the image
    idx = np.argsort(preds[i])[::-1][0]
    text = "Label: {}, {:.2f}%".format(classes[idx],
        preds[i][idx] * 100)
    cv2.putText(image, text, (5, 25),  cv2.FONT_HERSHEY_SIMPLEX,
        0.7, (0, 0, 255), 2)

    # display the output image
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

# Обнаружение объектов в режиме реального времени с глубоким обучением и OpenCV

In [7]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import imutils
import time
import cv2

In [8]:
confidence_min = 0.2
PATH_prototxt = '..\\src\\real-time-object-detection\\MobileNetSSD_deploy.prototxt.txt'
PATH_model = '..\\src\\real-time-object-detection\\MobileNetSSD_deploy.caffemodel'
PATH_video = '..\\src\\real-time-object-detection\\videos\\example_4.mp4'

# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))


In [9]:
# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(PATH_prototxt, PATH_model)

[INFO] loading model...


In [10]:
if PATH_video != None:
    vs = cv2.VideoCapture(PATH_video)
else:
    vs = cv2.VideoStream(src=0).start()
    time.sleep(2.0)

In [11]:
fps = FPS().start()

# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    ret, frame = vs.read()
    if frame is None:
        break
        
    frame = imutils.resize(frame, width=900)

    # grab the frame dimensions and convert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),
        0.007843, (300, 300), 127.5)

    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()

    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > confidence_min:
            # extract the index of the class label from the
            # `detections`, then compute the (x, y)-coordinates of
            # the bounding box for the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # draw the prediction on the frame
            label = "{}: {:.2f}%".format(CLASSES[idx],
                confidence * 100)
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

    # update the FPS counter
    fps.update()

# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
cv2.destroyAllWindows()
vs.release()

[INFO] elapsed time: 28.04
[INFO] approx. FPS: 20.25
